<H1> Notebook to verify the calculations of our simulator </H1>

In [ ]:
# preample
import qutip
import zipfile
import pickle
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy.signal import freqs,periodogram,cheby1
import numpy as np

In [ ]:
# verification parameters
dataset = "S_1q_X_Z_N4_D"
ex_num  = 1

In [ ]:
fname = "%s_ex_%d"%(dataset, ex_num)

# unzip the dataset zipfile and extract the example file
fzip  = zipfile.ZipFile("%s.zip"%dataset, mode='r')
fzip.extract( fname )
fzip.close()

# load the example file
f     = open(fname,  "rb")
data  = pickle.load(f)
f.close()
os.remove(fname)

In [ ]:
# plot the pulse
plt.figure()
num_controls = len(data["sim_parameters"]["dynamic_operators"])
for idx in range(num_controls):
    plt.subplot(num_controls , 1, idx+1 )
    plt.plot(data["time_range"], data["pulses"][0,:,0,idx], label="undistorted")
    plt.plot(data["time_range"], data["distorted_pulses"][0,:,0,idx], label="distorted")
    plt.xlabel('t')
    plt.ylabel('f(t)')
    plt.grid()
    plt.legend()
print(data["pulse_parameters"])

In [ ]:
# display distortion filter if exists
distortion = cheby1(4,0.1,2*np.pi*20, analog=True)
# evaluate frequency response of the filter
w, Hw = freqs(distortion[0], distortion[1])
plt.figure(figsize=[15,4])
plt.subplot(1,2,1)
plt.semilogx(w, 20*np.log(np.abs(Hw)))
plt.xlabel(r'$\Omega$')
plt.ylabel(r'$|H(\Omega)|$')
plt.grid()
plt.subplot(1,2,2)
plt.semilogx(w, np.angle(Hw))
plt.xlabel(r'$\Omega$')
plt.ylabel(r'arg $H(\Omega)$')
plt.grid()

In [ ]:
# display noise if exists
for idx_profile,profile in enumerate(data["sim_parameters"]["noise_profile"]): 
    if profile in [2,3,4] or (profile==6 and p==0): 
        # estimate the correlation matrix of the noise
        correlation = 0
        for k in range(data["sim_parameters"]["K"]):
            correlation = correlation + data["noise"][0,:,k:k+1,idx_profile]@data["noise"][0,:,k:k+1,idx_profile].T
        correlation = correlation/data["sim_parameters"]["K"]
        # plot correlation matrix
        plt.figure()
        plt.matshow(correlation,0)
        plt.colorbar()
        p = 0
    elif profile in [1,5]:
        # estimate the PSD of the noise
        psd = 0
        for k in range(data["sim_parameters"]["K"]):
            f, Pxx = periodogram(data["noise"][0,:,k,idx_profile], data["sim_parameters"]["M"]/data["sim_parameters"]["T"])            
            psd = psd + Pxx
        psd = psd/data["sim_parameters"]["K"]
        plt.figure()
        plt.plot(f[f>0], psd[1:])
        plt.xlabel('f')
        plt.ylabel('psd')
        plt.grid()
        p = 1

In [ ]:
# load initial states, measurement operators, and control Hamilotonian
initial_states = [qutip.Qobj(state) for state in data["sim_parameters"]["initial_states"] ] 
measurements   = [qutip.Qobj(op) for op in data["sim_parameters"]["measurement_operators"] ]

H0  = [ [qutip.Qobj(op), np.ones((len(data["time_range"])))] for op in data["sim_parameters"]["static_operators"] ] + [ [qutip.Qobj(op), data["distorted_pulses"][0,:,0,idx]] for idx, op in enumerate(data["sim_parameters"]["dynamic_operators"]) ]

expectations = np.zeros((1,data["sim_parameters"]["K"], len(initial_states)*len(measurements)))  
for idx_K in range(data["sim_parameters"]["K"]):    
    H1      = [ [qutip.Qobj(op), data["noise"][0,:,idx_K,idx]] for idx, op in enumerate(data["sim_parameters"]["noise_operators"]) ]
    results = [ qutip.mesolve(H0 + H1, rho, np.array(data["time_range"]), e_ops=measurements).expect for rho in initial_states]     
    expectations [0,idx_K, :] = np.concatenate( [np.array( [results[idx_rho][idx_M][-1] for idx_M in range(len(measurements))]) for idx_rho in range(len(initial_states))])
    print(idx_K+1,  end="\r")

In [ ]:
# plot the average expectation over all noise realizations for every observable
plt.figure()
plt.plot(np.average(expectations, 1)[0], label="qutip")
plt.plot(data["expectations"][0], label = "tf")
plt.ylabel("Average observable value")
plt.xlabel("observable Index")
plt.gca().xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
plt.legend()
plt.grid()

In [ ]:
# plot all possible observables for a particular noise realization
idx_K = 2
plt.figure()
plt.plot(expectations[0,idx_K,:], label="qutip")
plt.plot(data["Eo"][0,idx_K,:],  label = "tf")
plt.ylabel("Observable Value for realization %d"%idx_K)
plt.xlabel("Observable Index")
plt.gca().xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
plt.legend()
plt.grid()

In [ ]:
# simulation time without I/O operations
print("Total time for 1 batch is %f seconds"%data["sim_parameters"]["elapsed_time"])
print("Average time for 1 example %f seconds"%(data["sim_parameters"]["elapsed_time"]/ data["sim_parameters"]["batch_size"]))